# Test Summary
## Equipment
| Equipment | Description | Serial Number | Date of last calibration |
|:--|:--|:--|:--|
| Siglent SPD3303C| Labo DC Power supply| SPD3EEEC6R0513| 2022-03-06 |
| Siglent SDL1020X-E| DC Electronic Load | SDL13GCC6R0182| 2022-02-11|
| Mastech M92 | digital multimeter | 20011206033 | (?) |
| SDS2354X HD | oscilloscope | | |


# Test incident reports

## TIR1
* **Problem** : 
* **Proposed Fix** :


# Battery protection test
 <table>
  <tr>
    <th>Action</th>
    <th>Expected Result</th>
    <th>Observed result</th>
    <th>Status</th>
  </tr>
  <tr>
    <td><b>Power ON/OFF by PWREN</b> <br/>Set VBAT to 3.7V.  Enable power MCU.  Bringing PWREN high should close Q2, Q4, Q5</td>
    <td>V(BT1.2) = 0V</td>
    <td>V(BT1.2) = 0V</td>
    <td>✅</td>
  </tr>
  <tr>
    <td><b>Power ON/OFF by PWREN</b> <br/>Set VBAT to 3.7V.Disable power MCU.  Bringing PWREN low should open Q2, Q4, Q5</td>
    <td>V(BT1.2) > 1V</td>
    <td>V(BT1.2) > 1V</td>
    <td>✅</td>
  </tr>
  <tr>
    <td><b>Power ON/OFF by momentary switch</b> <br/>Set VBAT to 3.7V.Enable power MOM.  Bringing J2.1 to GND should close Q2, Q4, Q5</td>
    <td>V(BT1.2) = 0V</td>
    <td>V(BT1.2) = 0V</td>
    <td>✅</td>
  </tr>
  <tr>
    <td><b>Power ON/OFF by momentary switch</b> <br/>Set VBAT to 3.7V.Disable power MOM.  Disconnecting J2.1 should open Q2, Q4, Q5</td>
    <td>V(BT1.2) > 1V</td>
    <td>V(BT1.2) > 1V</td>
    <td>✅</td>
  </tr>
  <tr>
    <td><b>VOCP : over charge detection</b><br/>
    J2.1 connected to GND.<br/>
    Power supply connected to BT1.  Electronic load (3mA) connected on C7.<br/>
    Set VBAT to 3.7V.  Increase VBAT until V(BT1.2)>0V </td>
    <td>VOCP = 4.25 - 4.35V</td>
    <td>VOCP = 4.29V</td>
    <td>✅</td>
  </tr>
  <tr>
    <td><b>VOCR : over charge release</b><br/>
    J2.1 connected to GND.<br/>
    Power supply connected to BT1.  Electronic load (3mA) connected on C7.<br/>
    Set VBAT to 4.3V.  Decrease VBAT until V(BT1.2)=0V </td>
    <td>VOCR = 4.05 - 4.15V</td>
    <td>VOCR = 4.28V</td>
    <td>❌ >100mV out of spec, but should be no problem for this application</td>
  </tr>
  <tr>
    <td><b>VODP : Overdischarge detection voltage</b><br/>
    J2.1 connected to GND.<br/>
    Power supply connected to BT1.  Electronic load (3mA) connected on C7.<br/>
    Set VBAT to 3.7V.  Decrease VBAT until V(BT1.2)>0V </td>
    <td>VODP = 2.4V</td>
    <td>VODP = 2.51V</td>
    <td>❌ 100mV out of spec, but should be no problem for this application</td>
  </tr>
  <tr>
    <td><b>VODR : Overdischarge release voltage</b><br/>
    J2.1 connected to GND.<br/>
    Power supply connected to BT1.  Electronic load (3mA) connected on C7.<br/>
    J2.1 connected to GND.  Set VBAT to 2.4V.  Increase VBAT until V(BT1.2)=0V </td>
    <td>VODR = 3.0±0.100V</td>
    <td>VODR = 3.20V</td>
    <td>❌ 100mV out of spec, but should be no problem for this application</td>
  </tr>
  <tr>
    <td><b>Over current detection</b></td>
    <td></td>
    <td>3xFS8205A : 6.2A total → 93.6mV<br/>
    6.2A is the maximum my power supply can deliver.  MOSFETs would have to be removed to find the over current limit.</td>
    <td>❓Not tested</td>
  </tr>

</table> 


# Test result details
## Battery protection test
The problem with detection of over charge is that the voltage is measured over the battery.  When supplying current to the protected BATT+/BATT- terminals (symbol naming according to DW01A datasheet), the VCC/GND will be low because the two MOSFETS are open.  As such the device is in undervoltage lockout and charging can not begin.

When replacing the battery by a power supply and cranking up the voltage to overcharge, the problem is that the OC-MOSFET might be off, but its internal diode will still be conducting, so the current is not blocked.  We can check by measuring the drain-source voltage on the OC-MOSFET.  

So we can connect a unity gain buffer to instead of the battery and apply a current source to the BATT+/BATT- terminals.  By increasing the voltage on the buffer, we can simulate the battery voltage increasing.  When can then monitor the current source to see if the OC-MOSFET is conducting or not.